<a href="https://colab.research.google.com/github/Raul290697/TAREA1PROGRA/blob/main/ProyectoFinalProgra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
Este módulo entrena y compara diferentes clasificadores (Regresión Logística, KNN, SVM, Árbol de Decisión, Random Forest)
usando el conjunto de datos de cancer (cancer.xlsx).
El análisis consiste en:
- Lectura de datos
- Limpieza y separación de características y etiqueta
- División entre entrenamiento y validación
- Entrenamiento de múltiples clasificadores en una clase Clasificadores
- Evaluación con diferentes métricas y validación cruzada
- Comparación entre métodos y selección del mejor árbol de decisión
"""

import sys
import traceback
import logging

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns

# Configura el sistema para mostrar mensajes de error en la terminal

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')


class Clasificadores:
    """
    Clase que encapsula diversos clasificadores solicitados.
    Se almacenan como atributos:
    - Regresión Logística
    - KNN
    - SVM
    - Árbol de Decisión
    - Random Forest
    """
    def __init__(self, random_state=42):
        """
        Inicializa los clasificadores con algunos hiperparámetros por defecto.

        Parameters
        ----------
        random_state : int
            Semilla para la reproducibilidad.
        """
        self.log_reg = LogisticRegression(random_state=random_state, max_iter=1000)
        self.knn = KNeighborsClassifier(n_neighbors=5)
        self.svm = SVC(random_state=random_state, probability=True)
        self.decision_tree = DecisionTreeClassifier(random_state=random_state)
        self.random_forest = RandomForestClassifier(random_state=random_state)

    def entrenar_todos(self, X_train, y_train):
        """
        Entrena todos los clasificadores con los datos de entrenamiento.

        Parameters
        ----------
        X_train : np.ndarray
            Características de entrenamiento
        y_train : np.ndarray
            Etiquetas de entrenamiento
        """
        try:
            self.log_reg.fit(X_train, y_train)
            self.knn.fit(X_train, y_train)
            self.svm.fit(X_train, y_train)
            self.decision_tree.fit(X_train, y_train)
            self.random_forest.fit(X_train, y_train)
        except Exception as e:
            logging.error("Error al entrenar los clasificadores: %s", e)
            traceback.print_exc(file=sys.stdout)


def cargar_datos(ruta_archivo):
    """
    Carga el archivo xlsx con los datos de cáncer.

    Parameters
    ----------
    ruta_archivo : str
        Ruta al archivo .xlsx

    Returns
    -------
    df : pd.DataFrame
        DataFrame con los datos cargados.
    """
    try:
        df = pd.read_excel(ruta_archivo)
        return df
    except FileNotFoundError as fnf_error:
        logging.error("Archivo no encontrado: %s", fnf_error)
        sys.exit(1)
    except Exception as e:
        logging.error("Error al leer el archivo: %s", e)
        sys.exit(1)


def preparar_datos(df):
    """
    Prepara el conjunto de datos:
    - Elimina columna 'id'
    - Separa en X (características) e y (etiqueta)
    - Estandariza X
    - Convierte la columna diagnosis a valores binarios (ej. M=1, B=0)

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con todas las columnas.

    Returns
    -------
    X : np.ndarray
        Características escaladas.
    y : np.ndarray
        Etiquetas binarias.
    """
    try:
        # Verificar si existe la columna 'id' y eliminarla
        if 'id' in df.columns:
            df = df.drop(columns=['id'])

        # Suponemos que la columna 'diagnosis' es la etiqueta
        if 'diagnosis' not in df.columns:
            raise ValueError("La columna 'diagnosis' no está presente en el DataFrame.")

        # Codificar diagnosis: M=1, B=0
        df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})
        if df['diagnosis'].isnull().any():
            raise ValueError("La columna 'diagnosis' contiene valores no esperados.")

        y = df['diagnosis'].values
        X = df.drop(columns=['diagnosis']).values

        # Estandarización de X
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        return X_scaled, y

    except Exception as e:
        logging.error("Error en la preparación de datos: %s", e)
        sys.exit(1)


def evaluar_modelo(modelo, X_train, X_test, y_train, y_test):
    """
    Evalúa un modelo dado sobre conjuntos de entrenamiento y prueba,
    mostrando métricas como exactitud, matriz de confusión y F1-score.

    Parameters
    ----------
    modelo : objeto clasificador entrenado
    X_train : np.ndarray
        Datos de entrenamiento
    X_test : np.ndarray
        Datos de prueba
    y_train : np.ndarray
        Etiquetas de entrenamiento
    y_test : np.ndarray
        Etiquetas de prueba

    Returns
    -------
    resultados : dict
        Diccionario con métricas calculadas.
    """
    y_pred_train = modelo.predict(X_train)
    y_pred_test = modelo.predict(X_test)

    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)
    f1 = f1_score(y_test, y_pred_test)
    cm = confusion_matrix(y_test, y_pred_test)

    resultados = {
        'accuracy_train': acc_train,
        'accuracy_test': acc_test,
        'f1_test': f1,
        'confusion_matrix': cm
    }
    return resultados


def validacion_cruzada(modelo, X, y, cv=5):
    """
    Realiza validación cruzada con el modelo especificado.

    Parameters
    ----------
    modelo : objeto clasificador
        Modelo a validar.
    X : np.ndarray
        Datos de características.
    y : np.ndarray
        Etiquetas.
    cv : int
        Número de particiones para la validación cruzada.

    Returns
    -------
    mean_score : float
        Promedio de exactitud en la validación cruzada.
    std_score : float
        Desviación estándar de las puntuaciones.
    """
    scores = cross_val_score(modelo, X, y, cv=cv, scoring='accuracy')
    return np.mean(scores), np.std(scores)


if __name__ == "__main__":
    # Ruta del archivo (aquí puse el archivo en la misma carpeta)
    ruta = "/content/cancer.xlsx"

    # Cargar datos
    df = cargar_datos(ruta)

    # Preparar datos
    X, y = preparar_datos(df)

    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    # Crear instancia de la clase clasificadores
    clasificadores = Clasificadores(random_state=42)

    # Entrenar todos los clasificadores
    clasificadores.entrenar_todos(X_train, y_train)

    # Evaluar todos los clasificadores
    modelos = {
        'Logistic Regression': clasificadores.log_reg,
        'KNN': clasificadores.knn,
        'SVM': clasificadores.svm,
        'Decision Tree': clasificadores.decision_tree,
        'Random Forest': clasificadores.random_forest
    }

    # Se guardarán resultados para comparar
    resultados_globales = {}

    for nombre, modelo in modelos.items():
        res = evaluar_modelo(modelo, X_train, X_test, y_train, y_test)
        cv_mean, cv_std = validacion_cruzada(modelo, X, y, cv=5)
        res['cv_mean_acc'] = cv_mean
        res['cv_std_acc'] = cv_std
        resultados_globales[nombre] = res

    # resultados
    print("RESULTADOS DE LA EVALUACIÓN DE MODELOS:")
    for nombre, metrica in resultados_globales.items():
        print(f"\nModelo: {nombre}")
        print(f"Accuracy Entrenamiento: {metrica['accuracy_train']:.4f}")
        print(f"Accuracy Validación: {metrica['accuracy_test']:.4f}")
        print(f"F1-score Validación: {metrica['f1_test']:.4f}")
        print("Matriz de Confusión:")
        print(metrica['confusion_matrix'])
        print(f"Validación Cruzada (mean ± std): {metrica['cv_mean_acc']:.4f} ± {metrica['cv_std_acc']:.4f}")

    # Identificar cuál árbol de decisión fue el mejor:
    # En este caso, sólo entrenamos un árbol de decisión simple, así que la comparación
    # se haría entre el árbol de decisión y los demás clasificadores.


    # Supongamos que el árbol de decisión es el "mejor" si obtiene la mayor exactitud de validación.
    decision_tree_acc = resultados_globales['Decision Tree']['accuracy_test']
    # Comparar con los otros
    mejores_que_arbol = [m for m, r in resultados_globales.items() if r['accuracy_test'] > decision_tree_acc]

    print("\nCOMPARACIÓN FINAL:")
    if len(mejores_que_arbol) == 0:
        print("El Árbol de Decisión es el mejor modelo en términos de exactitud de validación.")
        print("Razón: Obtuvo la mayor exactitud en el conjunto de validación en comparación con otros métodos.")
    else:
        print("El Árbol de Decisión NO es el mejor modelo.")
        print("Otros modelos con mayor exactitud de validación:", mejores_que_arbol)
        print("Aunque el árbol de decisión tiene ventajas interpretables (facilidad de interpretación,")
        print("reducción del espacio de hipótesis), en este caso su desempeño de exactitud no superó")
        print("a otros clasificadores. Si quisiéramos el mejor árbol, podríamos ajustar hiperparámetros,")
        print("como la profundidad máxima, criterios de división, etc., para mejorar su rendimiento.")


def plot_confusion_matrix(model, X_test, y_test, model_name, save_path='images/'):
    """
    Genera y guarda una matriz de confusión para un modelo específico.

    Parameters:
    - model: Modelo entrenado.
    - X_test: Conjunto de características de prueba.
    - y_test: Etiquetas de prueba.
    - model_name: Nombre del modelo (para el título y el nombre del archivo).
    - save_path: Carpeta donde se guardará la imagen.
    """
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusión - {model_name}')
    plt.xlabel('Predicción')
    plt.ylabel('Realidad')
    plt.tight_layout()
    plt.savefig(f"{save_path}confusion_{model_name.replace(' ', '_').lower()}.png")
    plt.close()

# Generar matrices de confusión para cada modelo
modelos = {
    'Logistic Regression': clasificadores.log_reg,
    'KNN': clasificadores.knn,
    'SVM': clasificadores.svm,
    'Decision Tree': clasificadores.decision_tree,
    'Random Forest': clasificadores.random_forest,
}

for nombre, modelo in modelos.items():
    plot_confusion_matrix(modelo, X_test, y_test, nombre)


def plot_roc_curves(modelos, X_test, y_test, save_path='images/'):
    """
    Genera y guarda las curvas ROC para múltiples modelos.

    Parameters:
    - modelos: Diccionario con nombres y modelos entrenados.
    - X_test: Conjunto de características de prueba.
    - y_test: Etiquetas de prueba.
    - save_path: Carpeta donde se guardará la imagen.
    """
    plt.figure(figsize=(8, 6))
    for nombre, modelo in modelos.items():
        if hasattr(modelo, "predict_proba"):
            y_prob = modelo.predict_proba(X_test)[:, 1]
        else:
            y_prob = modelo.decision_function(X_test)
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{nombre} (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--', label='Línea Base (AUC = 0.50)')
    plt.xlabel('Tasa de Falsos Positivos')
    plt.ylabel('Tasa de Verdaderos Positivos')
    plt.title('Curvas ROC de los Modelos')
    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.savefig(f"{save_path}roc_curves.png")
    plt.close()

# Generar curva ROC
plot_roc_curves(modelos, X_test, y_test)

RESULTADOS DE LA EVALUACIÓN DE MODELOS:

Modelo: Logistic Regression
Accuracy Entrenamiento: 0.9914
Accuracy Validación: 0.9733
F1-score Validación: 0.9655
Matriz de Confusión:
[[90  2]
 [ 2 56]]
Validación Cruzada (mean ± std): 0.9800 ± 0.0126

Modelo: KNN
Accuracy Entrenamiento: 0.9771
Accuracy Validación: 0.9733
F1-score Validación: 0.9649
Matriz de Confusión:
[[91  1]
 [ 3 55]]
Validación Cruzada (mean ± std): 0.9720 ± 0.0039

Modelo: SVM
Accuracy Entrenamiento: 0.9885
Accuracy Validación: 0.9800
F1-score Validación: 0.9739
Matriz de Confusión:
[[91  1]
 [ 2 56]]
Validación Cruzada (mean ± std): 0.9740 ± 0.0185

Modelo: Decision Tree
Accuracy Entrenamiento: 1.0000
Accuracy Validación: 0.9267
F1-score Validación: 0.9060
Matriz de Confusión:
[[86  6]
 [ 5 53]]
Validación Cruzada (mean ± std): 0.9118 ± 0.0162

Modelo: Random Forest
Accuracy Entrenamiento: 1.0000
Accuracy Validación: 0.9533
F1-score Validación: 0.9381
Matriz de Confusión:
[[90  2]
 [ 5 53]]
Validación Cruzada (mean ± s